<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Firstly, I download all the dependencies needed:

In [2]:
#first I install geopy & folium
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.18.1-py_0       conda-forge

The following packages will be UPDATED:

    openssl:       1.0.2p-h470a237_1 conda-forge --> 1.0.2p-h470a237_2 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
ope

In [1]:
#then, I import the libraries needed
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import lxml.html as lh

print('Libraries imported.')

Libraries imported.


Now I get the table from the web page:

In [13]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)

doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')

col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [14]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from the table 
    if len(T)!=3:
        break
    
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

Now I make it a Pandas DataFrame:

In [15]:
Dict={title:column for (title,column) in col}
df1=pd.DataFrame(Dict)
df1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Now I remove rows where Borough is not assigned and reset the index, remove the \n from the Neighbourhood column, and fix the Postcode column name to fit the example:

In [16]:
df2 = df1[~df1['Borough'].isin(['Not assigned'])]
df3= df2.reset_index(drop=True)
df4=df3.rename(index=str, columns={"Postcode": "PostalCode"})
df=df4.rename(index=str, columns={"Neighbourhood\n": "Neighbourhood"})
df['Neighbourhood']= df['Neighbourhood'].str.lstrip('+-').str.rstrip('\n')
df.head(7)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [17]:
for i in df.index:
    if df['Neighbourhood'][i] == 'Not assigned':
        df.at[i, 'Neighbourhood'] = df['Borough'][i]
    else:
        df.at[i, 'Neighbourhood'] = df['Neighbourhood'][i]
df.head(7)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


Now I combine rows that have the same PostalCode but different neighborhoods

In [18]:
df = df.groupby(['PostalCode','Borough'], sort = False, as_index=False).agg(lambda x: ', '.join(x))
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [19]:
print ('The dataframe has', df.shape[0], 'rows')

The dataframe has 103 rows
